In [1]:
import requests
import pandas as pd

In [2]:
ENDPOINT_URL = "https://api.node.glif.io"

In [3]:
def filecoin_rpc(method: str, params):
    """
    Call a Filecoin JSON-RPC method with the given parameters.
    """
    payload = {
        "jsonrpc": "2.0",
        "method": method,
        "params": params,
        "id": 1,
    }
    response = requests.post(ENDPOINT_URL, json=payload).json()
    if "error" in response:
        raise Exception(response["error"])
    return response["result"]

In [4]:
chain_head = filecoin_rpc("Filecoin.ChainHead", [])
latest_tipset = filecoin_rpc(
    "Filecoin.ChainGetTipSetByHeight", [chain_head["Height"], None]
)
print(f"Latest epoch is {latest_tipset['Height']}")

Latest epoch is 3472632


In [5]:
state_miners = filecoin_rpc("Filecoin.StateListMiners", [chain_head["Cids"]])
print(f"Total number of miners is {len(state_miners)}")

Total number of miners is 612905


In [6]:
StateMinerSectors = pd.DataFrame(
    filecoin_rpc("Filecoin.StateMinerSectors", ["f01114587", None, chain_head["Cids"]])
)
StateMinerSectors.sample(5)

,SectorNumber,SealProof,SealedCID,DealIDs,Activation,Expiration,DealWeight,VerifiedDealWeight,InitialPledge,ExpectedDayReward,ExpectedStoragePledge,ReplacedSectorAge,ReplacedDayReward,SectorKeyCID,SimpleQAPower
60110,901448,8,{'/': 'bagboea4b5abcaimztuk5b4zfk647ym2xxzjri2...,None,1375386,3159720,0,0,147891627031854455,691015329331277,13593616660539049,0,0,None,False
51471,706607,8,{'/': 'bagboea4b5abcb4w3fcyj27skfregixatne5xrh...,None,1397924,3182867,0,0,148949142844069778,681659617724107,13531686513627846,0,0,None,False
77737,1101579,8,{'/': 'bagboea4b5abcarxa5j2gscomovfd474qpwndjc...,None,1620605,3342960,0,0,158764913569939844,594574059570375,11739598187102572,0,0,None,False
26698,400946,8,{'/': 'bagboea4b5abcaaeefztz7gxpeu2usmdevgzmwt...,None,1375623,3159720,0,0,147889104216986747,690870682561322,13585881471918062,0,0,None,False
114117,5015641,8,{'/': 'bagboea4b5abcac42e44cb37smjm7wq4fcbn6wi...,[55633165],3236503,4324334,0,37377588548599808,1899995191011097709,2146490162356217,42027255114188709,0,0,None,False


In [19]:
StateMinerPartitions = pd.DataFrame(
    filecoin_rpc(
        "Filecoin.StateMinerPartitions", ["f01114587", None, chain_head["Cids"]]
    )
)
StateMinerPartitions

,AllSectors,FaultySectors,RecoveringSectors,LiveSectors,ActiveSectors
0,"[3, 8, 1, 3, 1, 7, 1, 5, 1, 11, 2, 2, 1, 3, 1,...",[0],[0],[0],[0]
1,"[5014864, 3, 1, 28, 1, 2, 723, 2, 2, 2, 1, 24,...",[0],[0],"[5014864, 3, 1, 28, 1, 2, 723, 2, 2, 2, 1, 24,...","[5014864, 3, 1, 28, 1, 2, 723, 2, 2, 2, 1, 24,..."


In [7]:
StateMinerPower = pd.DataFrame(
    filecoin_rpc("Filecoin.StateMinerPower", ["f01114587", chain_head["Cids"]])
)
StateMinerPower

,MinerPower,TotalPower,HasMinPower
RawBytePower,3141407799771136,10149576889514065920,True
QualityAdjPower,31355404038176768,28283031234541748224,True


In [13]:
from fdp.db import query

query("select * from raw_storage_provider_daily_power limit 10")

CatalogException: Catalog Error: Table with name raw_storage_provider_daily_power does not exist!
Did you mean "public.raw_storage_providers_location_provider_quest"?

In [15]:
%pip install httpx --quiet


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
import httpx

r = httpx.get("https://httpbin.org/get")
r.json()

{'args': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'python-httpx/0.25.2',
  'X-Amzn-Trace-Id': 'Root=1-6579df8c-529f6562083d9b6c62d1b021'},
 'origin': '83.60.226.183',
 'url': 'https://httpbin.org/get'}

In [33]:
r.json()

{'jsonrpc': '2.0',
 'result': {'Cids': [{'/': 'bafy2bzacednou3obfgftqedzaw3uxqrht6kgsnlyari73myagihwsio2uwb4m'},
   {'/': 'bafy2bzacedjbwqotzvekzyha7wyq3brfn5fw64zkcnisyxrktep4cypode5ss'},
   {'/': 'bafy2bzacebsnhegv5tinhl4iedteaxtsa36ucaau2jcslpiacq6tonkfgkrgk'},
   {'/': 'bafy2bzacebkqjzd6ty4v5lhnmcpbttuvy4hgz4v2xygrysx4lsixu5phje7v4'}],
  'Blocks': [{'Miner': 'f01981603',
    'Ticket': {'VRFProof': 'gc/wabtcjVc9Pb9EQKZ3qw3NB/Qst1b63Aw1STtTLN5AGIjqmBTOHhmdOpBucY1SD0azhL03CG33JYslEMsBcyGB8WHr3/NiP92NyE+H9brH6w8nyZNNmYbxIdgyT3iv'},
    'ElectionProof': {'WinCount': 1,
     'VRFProof': 't8De6UsNotIXdQ7iMCShCb9/geB69LV9cB6a5eVOWdd0/23jcCnVsGGxv9JyO8S/Cbf+HaLdFy5K6LrmfC1xmcCqlI4ghLjHqJVPt+QCAakQ16zysTpO6pC7MuucFdgI'},
    'BeaconEntries': [{'Round': 3568507,
      'Data': 'ky457BzAo/jx2Y0FtMVt6/41rxn0EpK9AYWE9yyhpKKADIdI9ew6TQFyKHTaXwkqBIrVVnzpMGSBgUQQ+ygPkvJLi7Kdyzd1NXnAOK+ljik7n7hEI3vvVON4zoXQQzWF'}],
    'WinPoStProof': [{'PoStProof': 3,
      'ProofBytes': 'l65dqne4TlgOveM41NI+8foojBa

bafy2bzacednou3obfgftqedzaw3uxqrht6kgsnlyari73myagihwsio2uwb4m
bafy2bzacedjbwqotzvekzyha7wyq3brfn5fw64zkcnisyxrktep4cypode5ss
bafy2bzacebsnhegv5tinhl4iedteaxtsa36ucaau2jcslpiacq6tonkfgkrgk
bafy2bzacebkqjzd6ty4v5lhnmcpbttuvy4hgz4v2xygrysx4lsixu5phje7v4


In [36]:
chain_head["Height"]

3472632

In [38]:
f = []
async with httpx.AsyncClient() as client:
    for i in range(100):
        payload = {
            "jsonrpc": "2.0",
            "method": "Filecoin.ChainGetTipSetByHeight",
            "params": [chain_head["Height"] - i, None],
            "id": 1,
        }
        r = client.post(ENDPOINT_URL, json=payload)
        f.append(r)

In [69]:
import asyncio
import httpx


async def do_tasks():
    async with httpx.AsyncClient() as client:
        payload = {
            "jsonrpc": "2.0",
            "method": "Filecoin.ChainGetTipSetByHeight",
            "params": [chain_head["Height"], None],
            "id": 1,
        }
        tasks = [client.post(ENDPOINT_URL, json=payload) for i in range(10)]
        result = await asyncio.gather(*tasks)
        return result

In [51]:
a = do_tasks()

In [68]:
a

<coroutine object do_tasks at 0x7fc628f84c40>